In [50]:
import math
import copy
import numpy as np

In [172]:
class Node():
    
    def __init__(self, size, costs, sortedEdges, allSortedEdges, parent_constr, extra_constr=None):
    
        self.size = size # Number of cities
        self.costs = costs # Distance matrix
        self.sortedEdges = sortedEdges
        self.allSortedEdges = allSortedEdges
        self.extra_constr = extra_constr
        self.constraints = self.determine_constr(parent_constr)
        self.lowerBound = self.computeLowerBound()
        
    #This method calculates the simple lower bound (SB)
    #This is the sum over all vertices of the two smallest allowed edges
    def computeLowerBound(self):
        lb = 0
        for i in range(self.size):
            lower = 0
            t = 0
            for j in range(self.size):
                if self.constraints[i][j] == 1:
                    lower += self.costs[i][j]
                    t += 1
            tt = 0
            while t < 2:
                shortest = self.sortedEdges[i][tt]
                if self.constraints[i][shortest] == 2:
                    lower += self.costs[i][shortest]
                    t += 1
                tt += 1
                if tt >= self.size:
                    lower = math.inf
                    break
            lb += lower
        return lb

    #This method determines the constraints of a node 
    #based on the parent constaints and the extra constraint
    def determine_constr(self, parent_constr):
        constraints = copy.deepcopy(parent_constr)
        if self.extra_constr == None:
            return constraints
        fr = self.extra_constr[0]
        to = self.extra_constr[1]
        constraints[fr][to] = self.extra_constr[2]
        constraints[to][fr] = self.extra_constr[2]
        for i in range(2):
            constraints = self.removeEdges(constraints)
            constraints = self.addEdges(constraints)
        
        return constraints

    #50
    #This method exclude edges when:
    #1. Two other edges adjacent to the same vertex are included
    #2. Including the edge would create a subtour
    def removeEdges(self, constraints):
        for i in range(self.size):
            t = 0
            for j in range(self.size):
                if (i != j) and (constraints[i][j] == 1):
                    t += 1
            if t >= 2:
                for j in range(self.size):
                    if constraints[i][j] == 2:
                        constraints[i][j] = 0
                        constraints[j][i] = 0
        for i in range(self.size):
            for j in range(self.size):
                t = 1
                prev = i
                fr = j
                cycle = False
                nextOne = self.next_one(prev, fr, constraints)
                while (nextOne[0]):
                    t += 1
                    next = nextOne[1]
                    if next == i:
                        cycle = True
                        break
                    if t > self.size:
                        break
                    prev = fr
                    fr = next
                    nextOne = self.next_one(prev, fr, constraints)
                if (cycle) and (t < self.size) and (constraints[i][j] == 2):
                    constraints[i][j] = 0
                    constraints[j][i] = 0
        return constraints
    
    #This method checks if all but two edges adjacent to a vertex are excluded
    #If so, these edges are included
    def addEdges(self, constraints):
        for i in range(self.size):
            t = 0
            for j in range(self.size):
                if constraints[i][j] == 0:
                    t += 1
            if t == self.size - 2:
                for j in range(self.size):
                    if constraints[i][j] == 2:
                        constraints[i][j] = 1
                        constraints[j][i] = 1
        return constraints
    
    #Determines whether there exists an included edge that starts in fr and does not end in prev
    #If so, it also returns the endpoint of this edge.
    def next_one (self, prev, fr, constraints):
        for j in range(self.size):
            if (constraints[fr][j] == 1) and (j != prev):
                return [True, j]
        return [False]
    
    #Determines if a node representd a full tour by checking whether from every vertex
    #there are exactly 2 included edges and all other edges are excluded
    def isTour(self):
        for i in range(self.size):
            num_zero = 0
            num_one = 0
            for j in range(self.size):
                if self.constraints[i][j] == 0:
                    num_zero += 1
                elif self.constraints[i][j] == 1:
                    num_one += 1
            if (num_zero != self.size - 2) or (num_one != 2):
                return False
        return True

    #Checks if a node contains a subtour
    def contains_subtour(self):
        for i in range(self.size):
            next = self.next_one(i, i, self.constraints)
            if next[0]:
                prev = i
                fr = next[1]
                t = 1
                next = self.next_one(prev, fr, self.constraints)
            while next[0]:
                t += 1
                prev = fr
                fr = next[1]
                if (fr == i) and (t < self.size):
                    return True
                next = self.next_one(prev, fr, self.constraints)
                if t == self.size:
                    return False
        return False
    
    #Assumes the node represents a tour and returns the length
    def tourLength(self):
        length = 0
        fr = 0
        print ("1: ", self.next_one(fr, fr, self.constraints))
        if self.next_one(fr, fr, self.constraints)[0]:
            print ("2: ", self.next_one(fr, fr, self.constraints))
            to = self.next_one(fr, fr, self.constraints)[1]
            for i in range(self.size):
                length += self.costs[fr][to]
                temp = fr
                fr = to
                if self.next_one(temp, to, self.constraints)[0]:
                    to = self.next_one(temp, to, self.constraints)[1]
        return length

    #Determines the next constraint according to the branching strategy
    #lexicographic order (LG)
    def next_constraint(self):
        for i in range(self.size):
            for j in range(self.size):
                if self.constraints[i][j] == 2:
                    return [i,j]
                
    #If a node represents a tour
    #returns a string with the order of the vertices in the tour
    def __str__(self):
        if self.isTour():
            result = "0"
            fr = 0
            to = None
            for j in range(self.size):
                if self.constraints[fr][j] == 1:
                    to = j
                    result += "−" + str(j)
                    break
            for i in range(self.size - 1):
                for j in range(self.size):
                    if (self.constraints[to][j] == 1) and (j != fr):
                        result += "−" + str(j)
                        fr = to
                        to = j
                        break
            return result
        else:
            print("This node is not a tour")

In [173]:
#from Node import Node
import math
import time
import copy

class TSP():
    
    def __init__(self, size, costs, bestTour=math.inf):
        self.size = size
        self.costs = costs
        self.bestTour = bestTour
        self.bestNode = None
        self.bestNodeTime = 0
        self.num_createdNodes = 0
        self.num_prunedNodes = 0
        self.sortedEdges = self.sort_edges()
        self.allSortedEdges = self.sort_allEdges()
        
    def findSolution(self):
        root = self.create_root()
        self.num_createdNodes += 1
        T1 = time.perf_counter()
        self.BranchAndBound(root)
        T2 = time.perf_counter()
        print("−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−")
        print("The shortest tour is:", self.bestNode)
        print("It has a length of", self.bestTour, "km")
        print("Found in", T2-T1, "seconds")
        print("Best tour was found after:", self.bestNodeTime, "seconds")
        print("Number of nodes created: ", self.num_createdNodes)
        print("Number of nodes pruned: ", self. num_prunedNodes)
        
    #sorts the edges of the distance matrix per row returns matirix where each
    #row i contains the numbers 0 <= k <= (self.size-1) in increasing costs of the edges (i,k)
    def sort_edges(self):
        result = []
        for i in range(self.size):
            result.append([x for (y, x) in sorted(zip(self.costs[i], list(range(self.size))))])
        return result
    
    #sorts all edges of distance matrix
    #returns list of pairs [i,j] in order of increasing costs
    def sort_allEdges(self):
        edges = []
        lengths = []
        for i in range(self.size):
            for j in range(i+1, self.size):
                edges.append([i, j])
                lengths.append(self.costs[i][j])
        result = [z for (l, z) in sorted(zip(lengths, edges))]
        return result
    
    def create_root(self):
        no_constraints = []
        for i in range(self.size):
            row_i = []
            for j in range(self.size):
                if (i != j):
                    row_i.append(2)
                else:
                    row_i.append (0)
            no_constraints.append(row_i)
        root = Node(self.size, self.costs, self.sortedEdges, self.allSortedEdges, no_constraints)
        return root
    
    def BranchAndBound(self, node):
        if node.isTour():
            if node.tourLength() < self.bestTour:
                self.bestTour = node.tourLength()
                self.bestNode = node
                self.bestNodeTime = time.perf_counter()
                print("Found better tour: ", self.bestNode, "of length", self.bestTour, "km")
        else:
            new_constraint = copy.copy(node.next_constraint())
            new_constraint.append(1)
            leftChild = Node(self.size, self.costs, self.sortedEdges, self.allSortedEdges, node.constraints, new_constraint)
            new_constraint[2] = 0
            rightChild = Node(self.size, self.costs, self.sortedEdges, self.allSortedEdges, node.constraints, new_constraint)
            self.num_createdNodes += 2
            
            if self.num_createdNodes%401 == 0:
                print("Number of nodes created so far: ", self.num_createdNodes)
                print("Number of nodes pruned so far: ", self.num_prunedNodes)
            if self.num_createdNodes%51 == 0:
                print(".")
            if (leftChild.contains_subtour()) or (leftChild.lowerBound > 2 * self.bestTour):
                leftChild = None
                self.num_prunedNodes += 1
            if (rightChild.contains_subtour()) or (rightChild.lowerBound > 2 * self.bestTour):
                rightChild = None
                self.num_prunedNodes += 1
            if (leftChild != None) and (rightChild == None):
                self.BranchAndBound(leftChild)
            elif (leftChild == None) and (rightChild != None):
                self. BranchAndBound(rightChild)
            elif (leftChild != None) and (rightChild != None):
                if leftChild.lowerBound <= rightChild.lowerBound:
                    if leftChild.lowerBound < 2 * self.bestTour:
                        self.BranchAndBound(leftChild)
                    else:
                        leftChild = None
                        self.num_prunedNodes += 1
                    if rightChild.lowerBound < 2*self.bestTour:
                        self.BranchAndBound(rightChild)
                    else:
                        rightChild = None
                        self.num_prunedNodes += 1
                else:
                    if rightChild.lowerBound < 2*self.bestTour:
                        self.BranchAndBound(rightChild)
                    else:
                        rightChild = None
                        self.num_prunedNodes += 1
                    if leftChild.lowerBound < 2*self.bestTour:
                        self.BranchAndBound(leftChild)
                    else:
                        leftChild = None
                        self.num_prunedNodes += 1
       
    #Determines the next constraint using IL
    def next_constraint(self):
        for edge in self.allSortedEdges:
            i = edge[0]
            j = edge[1]
            if self.constraints[i][j] == 2:
                return edge
            
    #Calculates lower bound OT
    def computeLowerBound2 (self):
        lb = 0
        onetree = np.zeros((self.size, self.size), np.int8)
        t = 0
        for i in range(1, self.size):
            for j in range(i + 1, self.size):
                if self.constraints[i][j] == 1:
                    onetree[i][j] = 1
                    onetree[j][i] = 1
                    t += 1
                    lb += self.costs[i][j]
        for edge in self.allSortedEdges:
            if t >= self.size - 1:
                break
            i = edge[0]
            j = edge[1]
            if (self.constraints[i][j] == 2) and (i != 0):
                onetree[i][j] = 1
                onetree[j][i] = 1
            if self.onetree_contains_cycle(onetree):
                onetree[i][j] = 0
                onetree[j][i] = 0
            else:
                t += 1
                lb += self.costs[i][j]
        t = 0
        for j in range(self.size):
            if self.constraints[0][j] == 1:
                onetree[0][j] = 1
                onetree[j][0] = 1
                lb += self.costs[0][j]
                t += 1
        tt = 0
        while t < 2:
            shortest = self.sortedEdges[0][ tt]
            if self.constraints[0][shortest] == 2:
                onetree[0][shortest] = 1
                onetree[shortest][0] = 1
                lb += self.costs[0][shortest]
                t += 1
        tt += 1
        return lb

In [175]:
cost = [[0,141,118,171,126,69,158],
[141,0,226,34,212,208,82],
[118,226,0,232,56,107,194],
[171,34,232,0,200,233,63],
[126,212,56,200,0,105,145],
[69,208,107,233,105,0,212],
[158,82,194,63,145,212,0]]

In [179]:
T = TSP(20, cost)
T.findSolution()

.
.
1:  [True, 6]
2:  [True, 6]
1:  [True, 6]
2:  [True, 6]
Found better tour:  0−6−16−2−18−8−4−13−1−17−3−19−5−12−11−10−15−7−9−14−0 of length 437 km
.
1:  [True, 6]
2:  [True, 6]
.
Number of nodes created so far:  401
Number of nodes pruned so far:  103
.
.
1:  [True, 6]
2:  [True, 6]
1:  [True, 6]
2:  [True, 6]
Found better tour:  0−6−16−2−18−8−4−13−1−17−3−19−5−12−7−11−10−15−9−14−0 of length 433 km
.
.
.
.
.
.
Number of nodes created so far:  1203
Number of nodes pruned so far:  508
.
.
.
.
.
.
.
.
Number of nodes created so far:  2005
Number of nodes pruned so far:  918
.
.
.
.
.
.
.
.
Number of nodes created so far:  2807
Number of nodes pruned so far:  1324
.
.
.
.
.
.
.
Number of nodes created so far:  3609
Number of nodes pruned so far:  1724
.
.
.
.
.
.
.
.
Number of nodes created so far:  4411
Number of nodes pruned so far:  2135
.
.
.
.
1:  [True, 6]
2:  [True, 6]
1:  [True, 6]
2:  [True, 6]
Found better tour:  0−6−16−2−18−3−17−1−13−4−8−11−10−15−9−7−12−5−19−14−0 of length 432 

.
.
.
.
.
.
.
Number of nodes created so far:  52531
Number of nodes pruned so far:  26192
.
.
.
.
.
.
.
.
Number of nodes created so far:  53333
Number of nodes pruned so far:  26596
.
.
.
.
.
.
.
.
Number of nodes created so far:  54135
Number of nodes pruned so far:  26992
.
.
.
.
.
.
.
.
Number of nodes created so far:  54937
Number of nodes pruned so far:  27402
.
.
.
.
.
.
.
Number of nodes created so far:  55739
Number of nodes pruned so far:  27800
.
.
.
.
.
.
.
.
Number of nodes created so far:  56541
Number of nodes pruned so far:  28206
.
.
.
.
.
.
.
.
Number of nodes created so far:  57343
Number of nodes pruned so far:  28611
.
.
.
.
.
.
1:  [True, 6]
2:  [True, 6]
1:  [True, 6]
2:  [True, 6]
Found better tour:  0−6−2−18−16−9−7−15−10−11−17−1−13−8−4−3−12−5−19−14−0 of length 388 km
.
.
Number of nodes created so far:  58145
Number of nodes pruned so far:  29005
.
.
.
.
.
.
.
.
Number of nodes created so far:  58947
Number of nodes pruned so far:  29407
.
.
.
.
.
.
.
.
Number

.
.
.
.
.
.
.
Number of nodes created so far:  122305
Number of nodes pruned so far:  61100
.
.
.
.
.
.
.
.
Number of nodes created so far:  123107
Number of nodes pruned so far:  61501
.
.
.
.
.
.
.
.
Number of nodes created so far:  123909
Number of nodes pruned so far:  61890
.
.
.
.
.
.
.
.
Number of nodes created so far:  124711
Number of nodes pruned so far:  62295
.
.
.
.
.
.
.
.
Number of nodes created so far:  125513
Number of nodes pruned so far:  62700
.
.
.
.
.
.
.
Number of nodes created so far:  126315
Number of nodes pruned so far:  63105
.
.
.
.
.
.
.
.
Number of nodes created so far:  127117
Number of nodes pruned so far:  63505
.
.
.
.
.
.
.
.
Number of nodes created so far:  127919
Number of nodes pruned so far:  63913
.
.
.
.
.
.
.
.
Number of nodes created so far:  128721
Number of nodes pruned so far:  64287
.
.
.
.
.
.
.
.
Number of nodes created so far:  129523
Number of nodes pruned so far:  64694
.
.
.
.
.
.
.
.
Number of nodes created so far:  130325
Number o

.
.
.
.
.
.
.
.
Number of nodes created so far:  192881
Number of nodes pruned so far:  96375
.
.
.
.
.
.
.
.
Number of nodes created so far:  193683
Number of nodes pruned so far:  96782
.
.
.
.
.
.
.
.
Number of nodes created so far:  194485
Number of nodes pruned so far:  97184
.
.
.
.
.
.
.
.
Number of nodes created so far:  195287
Number of nodes pruned so far:  97595
.
.
.
.
.
.
.
Number of nodes created so far:  196089
Number of nodes pruned so far:  97979
.
.
.
.
.
.
.
.
Number of nodes created so far:  196891
Number of nodes pruned so far:  98380
.
.
.
.
.
.
.
.
Number of nodes created so far:  197693
Number of nodes pruned so far:  98784
.
.
.
.
.
.
.
.
Number of nodes created so far:  198495
Number of nodes pruned so far:  99185
.
.
.
.
.
.
.
.
Number of nodes created so far:  199297
Number of nodes pruned so far:  99580
.
.
.
.
.
.
.
.
Number of nodes created so far:  200099
Number of nodes pruned so far:  99990
.
.
.
.
.
.
.
.
Number of nodes created so far:  200901
Number

Number of nodes pruned so far:  130874
.
.
.
.
.
.
.
.
Number of nodes created so far:  262655
Number of nodes pruned so far:  131275
.
.
.
.
.
.
.
.
Number of nodes created so far:  263457
Number of nodes pruned so far:  131677
.
.
.
.
.
.
.
.
Number of nodes created so far:  264259
Number of nodes pruned so far:  132079
.
.
.
.
.
.
.
.
Number of nodes created so far:  265061
Number of nodes pruned so far:  132487
.
.
.
.
.
.
.
Number of nodes created so far:  265863
Number of nodes pruned so far:  132888
.
.
.
.
.
.
.
.
Number of nodes created so far:  266665
Number of nodes pruned so far:  133285
.
.
.
.
.
.
.
.
Number of nodes created so far:  267467
Number of nodes pruned so far:  133685
.
.
.
.
.
.
.
.
Number of nodes created so far:  268269
Number of nodes pruned so far:  134084
.
.
.
.
.
.
.
.
Number of nodes created so far:  269071
Number of nodes pruned so far:  134486
.
.
.
.
.
.
.
.
Number of nodes created so far:  269873
Number of nodes pruned so far:  134895
.
.
.
.
.
.
.

.
.
.
.
Number of nodes created so far:  331627
Number of nodes pruned so far:  165769
.
.
.
.
.
.
.
.
Number of nodes created so far:  332429
Number of nodes pruned so far:  166167
.
.
.
.
.
.
.
.
Number of nodes created so far:  333231
Number of nodes pruned so far:  166577
.
.
.
.
.
.
.
.
Number of nodes created so far:  334033
Number of nodes pruned so far:  166966
.
.
.
.
.
.
.
.
Number of nodes created so far:  334835
Number of nodes pruned so far:  167369
.
.
.
.
.
.
.
.
Number of nodes created so far:  335637
Number of nodes pruned so far:  167778
.
.
.
.
.
.
.
Number of nodes created so far:  336439
Number of nodes pruned so far:  168176
.
.
.
.
.
.
.
.
Number of nodes created so far:  337241
Number of nodes pruned so far:  168585
.
.
.
.
.
.
.
.
Number of nodes created so far:  338043
Number of nodes pruned so far:  168978
.
.
.
.
.
.
.
.
Number of nodes created so far:  338845
Number of nodes pruned so far:  169381
.
.
.
.
.
.
.
.
Number of nodes created so far:  339647
Numb

In [146]:
T = TSP(4, cost)
T.findSolution()

1:  [True, 1]
2:  [True, 1]
1:  [True, 1]
2:  [True, 1]
Found better tour:  0−1−3−2−0 of length 80 km
−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−
The shortest tour is: 0−1−3−2−0
It has a length of 80 km
Found in 0.0013992606845931732 seconds
Best tour was found after: 8180.851593548185 seconds
Number of nodes created:  5
Number of nodes pruned:  2


In [145]:
cost = [[0, 10, 15, 20],
       [10, 0, 35, 25],
       [15, 35, 0, 30],
       [20, 25, 30, 0]]

In [178]:
file = open("twenty.tsp", "r")
coordinates = np.genfromtxt(file, dtype = float, skip_header = 7, usecols = (1,2))

cost = np.array([[0]*len(coordinates)]*len(coordinates))
sortededges = np.array([[0]*len(coordinates)]*len(coordinates))
allsortededges = []
parentconstraint = np.array([[2]*len(coordinates)]*len(coordinates))

for i in range(len(coordinates)):
    for j in range(len(coordinates)):
        if i != j:
            cost[i][j] = np.sqrt(((coordinates[i][0]-coordinates[j][0])**2 + (coordinates[i][1]-coordinates[j][1])**2))
        else:
            cost[i][j] = 0
            parentconstraint[i][j] = 0
            
        if i > j:
            allsortededges += [cost[i][j]]
            
for k in range(len(coordinates)):
    sortededges[k] = sorted(cost[k])
        
allsortededges = sorted(allsortededges)
print (cost)
print (sortededges)
print (allsortededges)
print (parentconstraint)

[[  0  84  51  81 111  67  31  55 117  47  62  69  79  90  26  54  44  71
   69  60]
 [ 84   0  59  23  28  65  55  34  33  37  22  16  51   6  66  29  44  17
   51  65]
 [ 51  59   0  71  87  88  26  54  84  44  47  54  88  65  56  42  22  57
   19  82]
 [ 81  23  71   0  35  44  58  25  49  34  24  17  29  24  58  30  50  13
   69  46]
 [111  28  87  35   0  79  84  58  18  63  48  42  60  21  91  56  72  40
   76  81]
 [ 67  65  88  44  79   0  64  36  94  44  51  51  20  68  42  50  65  48
   94   7]
 [ 31  55  26  58  84  64   0  36  87  25  36  43  68  62  29  28  13  46
   40  58]
 [ 55  34  54  25  58  36  36   0  68  10  15  18  33  39  33  13  32  17
   59  34]
 [117  33  84  49  18  94  87  68   0  71  55  50  77  28 100  62  74  50
   71  96]
 [ 47  37  44  34  63  44  25  10  71   0  15  21  43  43  29   9  21  23
   50  40]
 [ 62  22  47  24  48  51  36  15  55  15   0   7  44  28  44   7  27  10
   47  49]
 [ 69  16  54  17  42  51  43  18  50  21   7   0  41  22  50  14

In [48]:
precursor = np.array([[(0,0)]*len(coordinates)]*len(coordinates))
#print(precursor)
                      
for i in range(len(coordinates)):
    for j in range(len(coordinates)):
        precursor[i][j] = [cost[i][j], j]
        


[[[ 0  0]
  [ 7  1]
  [44  2]
  [28  3]
  [44  4]
  [ 7  5]
  [27  6]
  [10  7]
  [47  8]
  [49  9]
  [53 10]]

 [[ 7  0]
  [ 0  1]
  [41  2]
  [22  3]
  [50  4]
  [14  5]
  [34  6]
  [ 3  7]
  [51  8]
  [50  9]
  [60 10]]

 [[44  0]
  [41  1]
  [ 0  2]
  [53  3]
  [54  4]
  [46  5]
  [65  6]
  [37  7]
  [91  8]
  [25  9]
  [91 10]]

 [[28  0]
  [22  1]
  [53  2]
  [ 0  3]
  [72  4]
  [35  5]
  [50  6]
  [22  7]
  [57  8]
  [69  9]
  [74 10]]

 [[44  0]
  [50  1]
  [54  2]
  [72  3]
  [ 0  4]
  [38  5]
  [38  6]
  [51  7]
  [69  8]
  [34  9]
  [52 10]]

 [[ 7  0]
  [14  1]
  [46  2]
  [35  3]
  [38  4]
  [ 0  5]
  [20  6]
  [17  7]
  [45  8]
  [47  9]
  [47 10]]

 [[27  0]
  [34  1]
  [65  2]
  [50  3]
  [38  4]
  [20  5]
  [ 0  6]
  [37  7]
  [31  8]
  [60  9]
  [26 10]]

 [[10  0]
  [ 3  1]
  [37  2]
  [22  3]
  [51  4]
  [17  5]
  [37  6]
  [ 0  7]
  [55  8]
  [48  9]
  [63 10]]

 [[47  0]
  [51  1]
  [91  2]
  [57  3]
  [69  4]
  [45  5]
  [31  6]
  [55  7]
  [ 0  8]
  [91  9]
  [2

In [32]:
Node = Node(11, cost, sortededges, sortededges, parentconstraint)

IndexError: index 26 is out of bounds for axis 0 with size 11

In [180]:
#Brute force algorithm
import itertools
import math

n = 20
distances = cost
minLength = math.inf
minTour = []

for tour in itertools.permutations(list(range(1, n))):
    fr = 0
    length = 0
    count = 0
    while count < n - 1:
        to = tour[count]
        length += distances[fr][to]
        fr = to
        count += 1
    length += distances[fr][0]
    if length < minLength:
        minLength = length
        minTour = tour
minTour = (0,) + minTour + (0,)
print("Shortest tour is: ", minTour)
print("It has a length of: ", minLength , "km")

KeyboardInterrupt: 